# 部署和使用 Chinese-LLaMA-Alpaca 进行中文问答

### SageMaker  Endpoint 部署模型
  
[Chinese-LLaMA-Alpaca](https://github.com/ymcui/Chinese-LLaMA-Alpaca): Chinese-LLaMA-Alpaca 基于 META AI 的 LLaMA 模型训练的支持中文的大语言模型。

首先，我们使用Sagemaker部署ChatGLM模型。
#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [ ]:
# !pip install --upgrade boto3
# !pip install --upgrade sagemaker

In [13]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

接下来，我们使用Sagemaker进行模型部署。

In [20]:
from sagemaker.pytorch.model import PyTorchModel

model_name = None
entry_point = 'inference_alpaca_7b.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

!touch dummy
!tar czvf model.tar.gz dummy
!rm -f dummy

model = PyTorchModel(
    name = model_name,
    model_data = "./model.tar.gz",
    entry_point = entry_point,
    source_dir = './code_cn_alpaca',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

dummy


In [27]:
endpoint_name = "chinese-alpaca-plus-7b"
instance_type = 'ml.g4dn.xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type,
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

----------!

In [29]:
def generate_prompt(text):
    return {"ask": f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{text}

### Response:"""}

In [30]:
%%time
predictor.predict(generate_prompt("天空为什么是蓝色的"))

CPU times: user 4.71 ms, sys: 0 ns, total: 4.71 ms
Wall time: 1.48 s


{'answer': '这是因为大气中的气体分子会散射太阳光，其中蓝色波长的光线更容易被散射。'}

In [18]:
inputs= [
    {"ask": "写一首关于交通信号灯的诗"},
    {"ask": "陨石为什么总能落在陨石坑里?" },
    {"ask": "为什么爸妈结婚没叫我参加婚礼?"}
]
response = predictor.predict(generate_prompt(inputs[0]))
print("\n\n问题: ", inputs[0]["ask"], "\n回答:\n", response["answer"])
response = predictor.predict(generate_prompt(inputs[1]))
print("\n\n问题: ", inputs[1]["ask"], "\n回答:\n", response["answer"])
response = predictor.predict(generate_prompt(inputs[2]))
print("\n\n问题: ", inputs[2]["ask"], "\n回答:\n", response["answer"])



问题:  写一首关于交通信号灯的诗 
回答:
 红绿相间，行人匆匆过；
绿灯亮起时，车流如梭飞。
黄灯鸣响处，车辆停下歇息；
红灯闪烁时，人们等待通行。


问题:  陨石为什么总能落在陨石坑里? 
回答:
 陨石之所以能够落入陨石坑，是因为它们受到重力的影响。当一颗小行星撞击地球时，它会释放出大量的能量和物质，其中一部分被吸收了，另一部分则形成了陨石坑。由于陨石的密度比周围的岩石要大得多，所以它们更容易被引力吸引到陨石坑中。


问题:  为什么爸妈结婚没叫我参加婚礼? 
回答:
 很抱歉，我无法回答你的问题。请理解我的能力和范围。


也可以通过Endpoint Name进行调用。

In [55]:
import json
import boto3

sagemaker_endpoint_name = endpoint_name

client = boto3.client('runtime.sagemaker')

def query_endpoint(query):
    encoded_json = json.dumps({"ask": query}).encode('utf-8')
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text

In [64]:
query_endpoint("给定一个文字输入，将其中的所有数字加1。\n“明天的会议在9点开始，记得准时到达。”\n")

'给定一个文字输入，将其中的所有数字加1。\n“明天的会议在9点开始，记得准时到达。”\n转换为数字后：“明天的会议在10点开始，记得准时到达。”'

### 删除Endpoint
如果部署的模型已经不使用，可以删除 Endpoint，避免不必要的开支。

In [26]:
predictor.delete_model()
predictor.delete_endpoint()

### 13B 模型
13B模型大小大概是23G，所以使用g5系列的实例，它的GPU是A10系列，现存是24GB。而且，我们需要用8-bit量化的方式部署，否则24GB的现存，也只是够部署的，但是在推理的时候，会出现OutOfMemory的错误。

In [6]:
from sagemaker.pytorch.model import PyTorchModel

model_name = None
entry_point = 'inference_alpaca.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

!touch dummy
!tar czvf model.tar.gz dummy
!rm -f dummy

model_13 = PyTorchModel(
    name = model_name,
    model_data = "./model.tar.gz",
    entry_point = entry_point,
    source_dir = './code_cn_alpaca',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

dummy


In [18]:
endpoint_name = "chinese-alpaca-plus-13b"
instance_type = 'ml.g5.2xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor_13 = model_13.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type,
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

---------!

由于13B的模型较大，使用具有24G显存的g5的机型进行部署，如果不进行量化，也无法部署成功，所以使用8bit方式量化部署，才能部署成功，但是推理的时间很长。

In [4]:
def generate_prompt(text):
    return {"ask": f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{text}

### Response:"""}

In [20]:
%%time
predictor_13.predict(generate_prompt("海为什么是蓝色的"))

CPU times: user 48.5 ms, sys: 0 ns, total: 48.5 ms
Wall time: 23.8 s


{'answer': '海水看起来是蓝色的是因为太阳光在穿过大气层时，会发生散射现象。当光线遇到水滴或气体分子时，一部分会被反射回来，而另一部分则被折射到不同的方向上。在这个过程中，短波长的蓝光更容易被分散和反射，因此我们看到的海水通常呈现出蓝色的颜色。 此外，海水中的微粒（如浮游生物、盐分等）也会对颜色产生影响，它们会使海水呈现绿色、棕色甚至红色等不同色调。 因此，海水的颜色并不是单一的原因所决定的，而是由多种因素共同作用的结果。'}

简单的推理，使用了近25秒。

In [6]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName="chinese-alpaca-plus-13b", ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


In [11]:
%%time
payload = generate_prompt("信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n")
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

国家名：阿根廷；人名：梅西。
CPU times: user 5.57 ms, sys: 0 ns, total: 5.57 ms
Wall time: 24.8 s


In [12]:
payload = generate_prompt("信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：")
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

阿根廷，梅西


In [13]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

信息抽取：
2022年世界杯的冠军是阿根廷队伍，梅西是MVP
问题：国家名，人名
答案：阿根廷，梅西 ->>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [ ]:
predictor_13.delete_model()
predictor_13.delete_endpoint()